In [5]:
import pandas as pd

In [6]:
df_results = pd.read_csv("data/local/stock_evaluation_results.csv", index_col=0)
selected_ticker = ['AAK','ABB','SAND','VOLV-B']

In [ ]:
df_test=df_results[['Lista','Sektor','ttm_momentum_clusterRank','ttm_momentum_clusterRank_Sektor_avg','pct_ch_3_m','pct_ch_3_m_Sektor_avg']].loc[selected_ticker]

In [84]:
def prepare_sector_comparison(df):
    df_sector_avg = df[['Sektor','ttm_momentum_clusterRank_Sektor_avg', 'pct_ch_3_m_Sektor_avg']].drop_duplicates()
    df_sector_avg['is_sektor'] = True
    df_sector_avg.rename(columns={
        'ttm_momentum_clusterRank_Sektor_avg': 'ttm_momentum_clusterRank',
        'pct_ch_3_m_Sektor_avg': 'pct_ch_3_m'
    }, inplace=True)
    df_sector_avg.set_index('Sektor', inplace=True, drop=False)
    df_sector_avg['Lista'] = df_sector_avg['Sektor'] + ' Genomsnitt'
    df = df.drop(columns=['ttm_momentum_clusterRank_Sektor_avg', 'pct_ch_3_m_Sektor_avg'])
    df['is_sektor'] = False
    return pd.concat([df, df_sector_avg], axis=0)



In [88]:
sektor_to_use = ['Industri','Teknik','Hälsovård']
df_scatter=df_results[df_results['Sektor'].isin(sektor_to_use)][['Lista','Sektor','ttm_momentum_clusterRank','ttm_momentum_clusterRank_Sektor_avg','pct_ch_3_m','pct_ch_3_m_Sektor_avg']]
df_scatter_to_use=prepare_sector_comparison(df_scatter)


In [89]:
df_scatter_to_use

,Lista,Sektor,ttm_momentum_clusterRank,pct_ch_3_m,is_sektor
ALIF-B,LargeCap,Hälsovård,65.8,24.050639,False
AZN,LargeCap,Hälsovård,13.2,9.915014,False
BIOA-B,LargeCap,Hälsovård,9.0,31.809867,False
BONEX,LargeCap,Hälsovård,2.6,-30.525648,False
CAMX,LargeCap,Hälsovård,2.3,-4.500381,False
...,...,...,...,...,...
XMR,FirstNorthPremier,Teknik,12.0,-22.972972,False
ZAPLOX,FirstNorthPremier,Teknik,65.7,10.975614,False
Hälsovård,Hälsovård Genomsnitt,Hälsovård,48.9,-11.622284,True
Industri,Industri Genomsnitt,Industri,52.2,1.297974,True


In [90]:
# scatter plot

import plotly.express as px
#import plotly.graph_objects as go

fig = px.scatter(df_scatter_to_use, 
                 x='ttm_momentum_clusterRank', 
                 y='pct_ch_3_m', 
                 #color=df_scatter_to_use['is_sektor'].map({True: 'Sector Average', False: 'Individual Stock'}),
                 color=df_scatter_to_use['Lista'],   
                 hover_data=['Sektor'],
                 hover_name=df_scatter_to_use.index)

# Add vertical and horizontal lines for sector averages
sector_data = df_scatter_to_use[df_scatter_to_use['is_sektor']]
for idx, row in sector_data.iterrows():
    x_val = row['ttm_momentum_clusterRank']
    y_val = row['pct_ch_3_m']
    
    # Vertical line
    fig.add_vline(x=x_val, line_dash="dash", line_color="red", opacity=0.5)
    # Horizontal line
    fig.add_hline(y=y_val, line_dash="dash", line_color="red", opacity=0.5)

fig.update_layout(title='TTM Momentum Cluster Rank vs 3-Month Percentage Change',
                  xaxis_title='TTM Momentum Cluster Rank',
                  yaxis_title='3-Month Percentage Change')
fig.show()